In [1]:
import sys
import os
from pathlib import Path

project_root = Path(os.getcwd()).parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

In [2]:
from summarization import FlanT5Trainer

/Users/komangwikananda/miniconda3/envs/sona-ai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/komangwikananda/miniconda3/envs/sona-ai/lib/python3.12/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/komangwikananda/miniconda3/envs/sona-ai/lib/python3.12/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <367D4265-B20F-34BD-94EB-4F3EE47C385B> /Users/komangwikananda/miniconda3/envs/sona-ai/lib/python3.12/site-packages/torchvision/image.so
  Reason: tried: '/Users/komangwikananda/miniconda3/envs/sona-ai/lib/python3.12/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/komangwikananda/miniconda3/envs/sona-ai/lib/python3.12/site-packages/torchvision/../../../libj

CUDA available: False


In [7]:
training_args = {
    # ===== TRAINING =====
    "num_train_epochs": 5,
    "learning_rate": 2e-5,
    "per_device_train_batch_size": 4,
    "per_device_eval_batch_size": 4,
    "weight_decay": 0.01,
    "gradient_accumulation_steps": 4,

    # ===== EVALUATION / LOGGING / SAVING =====
    "evaluation_strategy": "epoch",
    "logging_strategy": "epoch",
    "save_strategy": "epoch",

    # ===== SEQ2SEQ SPECIFIC =====
    "predict_with_generate": True,
    "generation_max_length": 256,
    "generation_num_beams": 4,

    # ===== PERFORMANCE =====
    "fp16": False,
    "gradient_checkpointing": True,

    # ===== MODEL SELECTION =====
    "load_best_model_at_end": True,
    "metric_for_best_model": "rougeL",
    "greater_is_better": True,

    # ===== MISC =====
    "report_to": "none",

    # REQUIRED
    "output_dir": "../outputs/summarization",
}


trainer = FlanT5Trainer(config="flan-t5")
trainer.prepare_data_and_trainer(
    training_args_dict=training_args,
    train_size=100,
    val_size=10,
    test_size=10
)

Applying new LoRA configuration...
trainable params: 9,437,184 || all params: 792,587,264 || trainable%: 1.1907
Preparing dataset...
Loading processed dataset from disk...
Merging custom training arguments...


In [ ]:
trainer.train()

In [ ]:
trainer.test()

# Inference

In [2]:
from summarization import FlanT5Inferencer

c:\Users\nanda\miniconda3\envs\sona-ai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA available: True
CUDA device: NVIDIA GeForce RTX 3060 Laptop GPU


In [3]:
text = f"Nik: I don't think we should apply this change. \nRen: I agree. The helmet make the side ears looks too small. The face proportion looks weird. \nNik: But, I like the helmet tho. It looks kinda nice. \nRen: Should I redraw and adjust this? So we can keep it. \nNik: Yeah, I really think wearing a helmet will make this character looks awesome. It is just this helmet is not suitable for him. But, the design is cool tho. Maybe for the other character. \nRen: Yeah, sure. I will redraw it, then I will comeback to you later. \nNik: Ok, sure."
print(text)

Nik: I don't think we should apply this change. 
Ren: I agree. The helmet make the side ears looks too small. The face proportion looks weird. 
Nik: But, I like the helmet tho. It looks kinda nice. 
Ren: Should I redraw and adjust this? So we can keep it. 
Nik: Yeah, I really think wearing a helmet will make this character looks awesome. It is just this helmet is not suitable for him. But, the design is cool tho. Maybe for the other character. 
Ren: Yeah, sure. I will redraw it, then I will comeback to you later. 
Nik: Ok, sure.


In [5]:
inferencer = FlanT5Inferencer(
    config_name="flan-t5",
    use_pretrained=True,
    device="cpu",
    max_new_tokens=256,
    num_beams=4,
)
summary = inferencer.generate(text)
print(summary)

Using pretrained LoRA adapters from d:\Jobs\sona-ai\cp\flan-t5-small-mediasum...
Nik and Ren will redraw the helmet.
